1. INICION DE SESIÓN SOBRE SAGE MAKER

In [12]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Verificamos
print(region)

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
us-east-1


2. LECTURA DE ARCHIVOS DE LA VALIDACION CRUZADA

In [13]:
#Utilitario para leer archivos de datos
from sagemaker.inputs import TrainingInput

#Bucket en donde se encuentran los archivos
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdajac"

#Lectura de datos de entrenamiento
dataTrain = TrainingInput(
    f"s3://{bucket}/data/insurance_dataset_validacion_cruzada/vc1/train/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

#Lectura de datos de validación
dataTest = TrainingInput(
    f"s3://{bucket}/data/insurance_dataset_validacion_cruzada/vc1/test/", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

3.  CONFIGURACION DEL ALGORITMO 

In [14]:
#Importamos el utilitario para definir el entrenador del algoritmo
from sagemaker.estimator import Estimator

#Definimos el entrenador del algoritmo
entrenador = Estimator(
    image_uri = sagemaker.image_uris.retrieve("linear-learner", region), #Descargamos la implementación del algoritmo desde la región donde trabajamos
    role = rol, #Rol que ejecuta servicios sobre AWS
    instance_count = 1, #Cantidad de servidores de entrenamiento
    instance_type = "ml.m5.large", #Tipo de servidor de entrenamiento
    predictor_type = "regressor", #Tipo de predicción del algoritmo
    sagemaker_session = sesion, #Sesión de SageMaker
    base_job_name = "entrenamiento-prediccion-numerica-vc1" #Nombre del job de entrenamiento
)

#Configuramos los parametros del algoritmo
entrenador.set_hyperparameters(
    feature_dim = 11, #Cantidad de features
    predictor_type = "regressor", #Indicamos que tipo de predicción es
    normalize_data = "true", #Normalizamos los features
    normalize_label = "true" #Normalizamos el label
)

4. DEFINICION DE HIPERPARAMETROS 

In [15]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores continuos
from sagemaker.tuner import ContinuousParameter

In [16]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores enteros
from sagemaker.tuner import IntegerParameter

In [17]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores enteros
from sagemaker.tuner import CategoricalParameter

In [18]:
#Definimos los hyper-parámetros del modelo
hyperparametros = {
    "learning_rate": CategoricalParameter([0.0001, 0.001, 0.01, 0.1]), #Ratio de aprendizaje
    "l1": CategoricalParameter([0.001, 0.01, 0.1])  #Tamaño de batch
}

In [19]:
#Utilitario para crear la malla de hyper-parametrización
from sagemaker.tuner import HyperparameterTuner

In [20]:
#Creamos la malla de hyper-parametrización
mallaDeHyperParametros = HyperparameterTuner(
    entrenador, #Entrenador del algoritmo
    "validation:mse", #Métrica para elegir el mejor modelo
    hyperparametros, #Hyper-parámetros definidos
    objective_type = "Minimize", #Elegimos el modelo con el mayor valor de accuracy
    max_jobs = 12, #Cantidad de procesos de entrenamiento.
    max_parallel_jobs = 10 #Cantidad de procesos de entrenamiento en paralelo
)

5. ENTRENAMIENTO DEL MODELO 

In [21]:
#Ejecutamos la malla de hyper-parametrización
#TIEMPO: 5 MINUTOS
mallaDeHyperParametros.fit(inputs = {"train": dataTrain, "validation": dataTest})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


....................................................!


5. SELECCION DEW MEJOR MODELO

In [22]:
#Librería de AWS
import boto3

#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

In [23]:
#Obtenemos el mejor modelo entrenado (el del menor error)
nombreDelMejorModelo = mallaDeHyperParametros.best_training_job()

In [24]:
#Verificamos
print(nombreDelMejorModelo)

linear-learner-240927-1556-012-966d28e8


7. ESTADISTICAS DEL MODELO

In [25]:
#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDelMejorModelo)

#Dentro de la sub-variable "FinalMetricDataList" tenemos las métricas del modelo
descripcionDeEntrenamiento["FinalMetricDataList"]

[{'MetricName': 'train:progress',
  'Value': 100.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricName': 'validation:mae',
  'Value': 3458.134521484375,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricName': 'train:objective_loss',
  'Value': 0.3640965223312378,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricName': 'validation:objective_loss',
  'Value': 41873456.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricName': 'validation:objective_loss:final',
  'Value': 31685344.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricName': 'validation:rmse',
  'Value': 5628.9736328125,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricName': 'validation:mse',
  'Value': 31685344.0,
  'Timestamp': datetime.datetime(2024, 9, 27, 16, 0, 32, tzinfo=tzlocal())},
 {'MetricNam

In [26]:
#Imprimimos sólo la métrica de nuestro interés
for metrica in descripcionDeEntrenamiento["FinalMetricDataList"]:
    if metrica["MetricName"] == "validation:mse":
        print(metrica["Value"])

31685344.0


In [27]:
#Extraemos los hyper-parametros del modelo
descripcionDeEntrenamiento["HyperParameters"]

{'_tuning_objective_metric': 'validation:mse',
 'feature_dim': '11',
 'l1': '0.01',
 'learning_rate': '0.1',
 'normalize_data': 'true',
 'normalize_label': 'true',
 'predictor_type': 'regressor'}

In [28]:
#Extraemos el learning rate
descripcionDeEntrenamiento["HyperParameters"]["learning_rate"]

'0.1'

In [29]:
#Extraemos el batch size
descripcionDeEntrenamiento["HyperParameters"]["l1"]

'0.01'